# Import libiraries

In [1]:
# Import libraries
import warnings
warnings.filterwarnings("ignore")
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential,Model
from keras.layers.convolutional import Conv2D,MaxPooling2D
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from keras import layers
from keras import optimizers

# Reading the images

In [2]:
# gray scale image list
img_data_list=[]

# RGB image list
rgb_images = []

# List for labels 
labels = []

myList = os.listdir("/kaggle/input/sign-language-digits/Dataset")

for i in myList:
    data_dir_list =  os.listdir('/kaggle/input/sign-language-digits/Dataset/'+ str(i) )
    for img in data_dir_list:
            input_img=cv2.imread('/kaggle/input/sign-language-digits/Dataset/' + str(i)+ '/'+ '/'+ img)
            rgb_img_resize=cv2.resize(input_img,(100,100))
            rgb_images.append(rgb_img_resize)
            input_img= cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
            input_img_resize=cv2.resize(input_img,(100,100))
            img_data_list.append(input_img_resize)
            labels.append(i)   

# Size of rgb image

In [3]:
rgb_images[0].shape

(100, 100, 3)

# Size After convert images to gray scale

In [4]:
img_data_list[0].shape

(100, 100)

# Scaling for gray scale images /255

In [5]:
# convert img_data_list into np.array
img_data_list = np.array(img_data_list)
img_data_list = img_data_list.astype('float32')


# Scaling for gray scale images /255
img_data_list /= 255.0

# convert labels into np.array
labels = np.array(labels).astype('int32')

# Building 2 neural network

# First neural network

In [6]:
def model():
    model = Sequential();
    
    model.add(Flatten(input_shape=(100,100)))
    
    model.add(Dense(256,activation="relu"))
    model.add(Dropout(0.1))
    
    model.add(Dense(256,activation="relu"))
    model.add(Dropout(0.1))
     
    
    model.add(Dense(128,activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(10,activation="softmax"))
    
    model.compile(Adam(lr=0.00001),loss="sparse_categorical_crossentropy",metrics=['accuracy'])

    return model
  
model1=model()

2022-12-31 13:55:29.110993: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


# First neural network cross validation fit

In [7]:
train_acc1 = []
test_acc1 = []
print("********************First neural network(ANN)***************************")
kf = KFold(n_splits=5,random_state=None, shuffle=True)
i = 1
for train_index, test_index in kf.split(img_data_list):
    X_train, X_test = img_data_list[train_index], img_data_list[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    print("Cross Validation Fold Number",str(i))
    model1.fit(X_train,y_train,epochs=100,batch_size=128,max_queue_size=20,use_multiprocessing=True,workers=1,verbose=0)
    i +=1
    y_pred = model1.predict(X_test)
    y_pred =np.argmax(y_pred, axis=1)
    train_eval = model1.evaluate(X_train,y_train,verbose=0)
    test_eval = model1.evaluate(X_test,y_test,verbose=0)
    print("Train accuracy : ",train_eval[1]*100)
    print("Test accuracy : ",test_eval[1]*100)
    print(classification_report(y_pred,y_test))
    train_acc1.append(train_eval[1])
    test_acc1.append(test_eval[1])
    print("******************************************************")

print("Summary of First NN: ")
print("Mean Train accuracy: ",np.mean(train_acc1)*100)
print("Mean Test accuracy: ",np.mean(test_acc1)*100)

********************First neural network(ANN)***************************
Cross Validation Fold Number 1


2022-12-31 13:55:29.332346: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Train accuracy :  70.9520936012268
Test accuracy :  67.07021594047546
              precision    recall  f1-score   support

           0       0.87      0.74      0.80        46
           1       0.63      0.82      0.71        39
           2       0.65      0.57      0.61        49
           3       0.77      0.80      0.78        45
           4       0.44      0.44      0.44        32
           5       0.78      0.75      0.76        52
           6       0.47      0.58      0.52        31
           7       0.62      0.46      0.52        46
           8       0.67      0.63      0.65        41
           9       0.72      0.91      0.81        32

    accuracy                           0.67       413
   macro avg       0.66      0.67      0.66       413
weighted avg       0.68      0.67      0.67       413

******************************************************
Cross Validation Fold Number 2
Train accuracy :  79.1388750076294
Test accuracy :  76.27118825912476
              p

# Second neural network

In [8]:
def model():
    model = Sequential();
    model.add(Flatten(input_shape=(100,100)))
    model.add(Dense(256,activation="relu"))
    model.add(Dropout(0.1))

    model.add(Dense(128,activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(10,activation="softmax"))
    model.compile(Adam(lr=0.0001),loss="sparse_categorical_crossentropy",metrics=['accuracy'])

    return model
  
model2=model()

# Second neural network cross validation fit

In [9]:
train_acc2 = []
test_acc2 = []
print("********************Second neural network(ANN)**********************")
kf = KFold(n_splits=5,random_state=None, shuffle=True)
i = 1
for train_index, test_index in kf.split(img_data_list):
    X_train, X_test = img_data_list[train_index], img_data_list[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    print("Cross Validation Fold Number",str(i))
    model2.fit(X_train,y_train,epochs=100,batch_size=256,max_queue_size=20,use_multiprocessing=True,workers=1,verbose=0)
    i +=1
    y_pred = model2.predict(X_test)
    y_pred =np.argmax(y_pred, axis=1)
    train_eval = model2.evaluate(X_train,y_train,verbose=0)
    test_eval = model2.evaluate(X_test,y_test,verbose=0)
    print("Train accuracy : ",train_eval[1]*100)
    print("Test accuracy : ",test_eval[1]*100)
    print(classification_report(y_pred,y_test))
    train_acc2.append(train_eval[1])
    test_acc2.append(test_eval[1])
    print("******************************************************")
                                                                 

print("Summary of Second NN: ")
print("Mean Train accuracy: ",np.mean(train_acc2)*100)
print("Mean Test accuracy: ",np.mean(test_acc2)*100)

********************Second neural network(ANN)**********************
Cross Validation Fold Number 1
Train accuracy :  84.2328667640686
Test accuracy :  76.27118825912476
              precision    recall  f1-score   support

           0       0.84      0.89      0.86        36
           1       0.91      0.89      0.90        35
           2       0.79      0.58      0.67        52
           3       0.87      0.89      0.88        37
           4       0.64      0.65      0.65        46
           5       0.98      0.82      0.89        55
           6       0.67      0.68      0.67        41
           7       0.57      0.66      0.61        41
           8       0.74      0.78      0.76        36
           9       0.69      0.91      0.78        34

    accuracy                           0.76       413
   macro avg       0.77      0.77      0.77       413
weighted avg       0.77      0.76      0.76       413

******************************************************
Cross Validation

# conclusion:
- Second arciticture is better than the first
> so we run the second NN on the entire Train Data to see the accuracy

# Rerun the second arciticuter on the entire training Data 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(img_data_list, labels, test_size=0.2,shuffle=True)
arch2 = model2.fit(X_train,y_train,epochs=100,batch_size=32,
                    max_queue_size=20,
                    use_multiprocessing=True,
                    workers=1,verbose=0)

train_eval = model2.evaluate(X_train,y_train,verbose=0)
test_eval = model2.evaluate(X_test,y_test,verbose=0)
print("Train accuracy : ",train_eval[1])
print("Test accuracy : ",test_eval[1])
y_pred = model2.predict(X_test)
y_pred =np.argmax(y_pred, axis=1)
print(classification_report(y_pred,y_test))

Train accuracy :  0.9963614344596863
Test accuracy :  0.8983050584793091
              precision    recall  f1-score   support

           0       0.98      0.89      0.93        45
           1       0.98      1.00      0.99        43
           2       0.76      1.00      0.87        29
           3       0.97      1.00      0.98        31
           4       0.91      0.72      0.80        54
           5       0.96      1.00      0.98        46
           6       0.63      0.93      0.75        29
           7       1.00      0.75      0.85        51
           8       0.86      0.95      0.90        40
           9       0.95      0.89      0.92        45

    accuracy                           0.90       413
   macro avg       0.90      0.91      0.90       413
weighted avg       0.91      0.90      0.90       413



# Train using normalized RGB image 

In [11]:
rgb_images = np.array(rgb_images)
rgb_images.shape

(2062, 100, 100, 3)

# Preprocessing and normalization of Rgb images

In [12]:
Average = np.mean(rgb_images,axis=0)
for i in range(len(rgb_images)):
    rgb_images[i] = rgb_images[i] - Average

In [13]:
rgb_images = rgb_images.astype('float32')
rgb_images /= 255.0
print (rgb_images.shape)

(2062, 100, 100, 3)


# A Convolutional Neural Network CNN

In [14]:
def model():
    model = Sequential()
    model.add(layers.Convolution2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Convolution2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Convolution2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
       
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(Adam(lr=0.0001),loss="sparse_categorical_crossentropy",metrics=['accuracy'])

    return model

model3=model()

# CNN cross validation fit

In [15]:
train_acc3 = []
test_acc3 = []
print("*******************************CNN*******************************")
kf = KFold(n_splits=5,random_state=None, shuffle=True)
i = 1
for train_index, test_index in kf.split(rgb_images):
    X_train, X_test = rgb_images[train_index], rgb_images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    print("Cross Validation Fold Number",str(i))
    model3.fit(X_train,y_train,epochs=25,batch_size=256,max_queue_size=20,use_multiprocessing=True,workers=1,verbose=0)
    i +=1
    y_pred = model3.predict(X_test)
    y_pred =np.argmax(y_pred, axis=1)
    train_eval = model3.evaluate(X_train,y_train,verbose=0)
    test_eval = model3.evaluate(X_test,y_test,verbose=0)
    print("Train accuracy : ",train_eval[1]*100)
    print("Test accuracy : ",test_eval[1]*100)
    print(classification_report(y_pred,y_test))
    train_acc3.append(train_eval[1])
    test_acc3.append(test_eval[1])
    print("******************************************************")


print("Summary of CNN: ")
print("Mean Train accuracy: ",np.mean(train_acc3)*100)
print("Mean Test accuracy: ",np.mean(test_acc3)*100)

*******************************CNN*******************************
Cross Validation Fold Number 1
Train accuracy :  79.4420838356018
Test accuracy :  74.57627058029175
              precision    recall  f1-score   support

           0       1.00      0.64      0.78        53
           1       0.78      0.85      0.81        41
           2       0.62      0.80      0.70        35
           3       0.84      0.91      0.88        35
           4       0.62      0.52      0.56        46
           5       0.76      0.92      0.83        38
           6       0.68      0.72      0.70        53
           7       0.60      0.62      0.61        29
           8       0.77      0.77      0.77        39
           9       0.83      0.77      0.80        44

    accuracy                           0.75       413
   macro avg       0.75      0.75      0.74       413
weighted avg       0.76      0.75      0.74       413

******************************************************
Cross Validation Fo

# Using support verctor machine  (SVC)

> convert images to flatten array

In [16]:
flat_images = []
for i in range(len(rgb_images)):
    flat_images.append(rgb_images[i].flatten())
flat_images = np.array(flat_images)

# Split data after flatten to run svm

In [17]:
x = pd.DataFrame(flat_images)
y = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=15000,shuffle=True)

In [18]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1649, 30000)
(1649,)
(413, 30000)
(413,)


In [19]:
svm = SVC()
svm.fit(X_train, y_train)

train_acc = svm.score(X_train, y_train)
print('Model Train accuracy is: ', train_acc)

test_acc = svm.score(X_test, y_test)
print('Model Test accuracy is: ', test_acc)

print(classification_report(y_test, svm.predict(X_test)))

Model Train accuracy is:  0.9666464523953912
Model Test accuracy is:  0.6368038740920097
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        41
           1       0.71      0.66      0.68        41
           2       0.45      0.61      0.52        36
           3       0.84      0.63      0.72        43
           4       0.47      0.39      0.42        44
           5       0.91      0.78      0.84        40
           6       0.53      0.62      0.57        42
           7       0.37      0.61      0.46        33
           8       0.69      0.69      0.69        42
           9       0.79      0.59      0.67        51

    accuracy                           0.64       413
   macro avg       0.66      0.64      0.64       413
weighted avg       0.67      0.64      0.65       413



# End 